# Data Preparation - Dataset de Sepsis
## CRISP-DM Fase 3: Preparação dos Dados

**Objetivo da Fase:**
* Transformar dados brutos em formato adequado para modelagem
* Implementar estratégias de limpeza e tratamento baseadas nos insights da EDA
* Criar features derivadas com relevância clínica
* Preparar datasets finais para algoritmos de machine learning

**Baseado nos Insights da EDA:**
* 37/41 variáveis apresentam missing values (68.37% do dataset)
* 27 variáveis com >80% missing (candidatas à remoção)
* Dataset altamente desbalanceado: 98.2% não-sepsis vs 1.8% sepsis
* Estrutura temporal importante: risco aumenta após 100h na UTI
* Variáveis categóricas bem definidas: Gender, Unit1, Unit2

**Tarefas CRISP-DM a serem executadas:**
1. **Seleção dos Dados**: Escolher variáveis mais relevantes
2. **Limpeza dos Dados**: Tratar inconsistências e valores ausentes  
3. **Construção dos Dados**: Criar features derivadas e engenharia
4. **Integração dos Dados**: Combinar fontes (não aplicável aqui)
5. **Formatação dos Dados**: Preparar formato final para modelagem

## Configuração do Ambiente Google Colab

Para funcionar no Google Colab, é necessário criar um atalho do diretório MDA no seu próprio Drive e então rodar os dois comandos abaixo e conceder permissão ao seu drive quando rodar a célula logo abaixo.

[Link](https://towardsdatascience.com/simplify-file-sharing-44bde79a8a18/) detalhando como funciona

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# modificar para o diretorio que contém os dados de teste e treino
%cd /content/drive/MyDrive/MDA/Train\ and\ test\ data\ -\ Proj\ DM/

!ls

## 1. Importação das Bibliotecas

Importação de todas as bibliotecas necessárias para preparação dos dados, incluindo bibliotecas específicas para pré-processamento, feature engineering e balanceamento de classes.

In [ ]:
# Bibliotecas essenciais para manipulação de dados
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Bibliotecas para pré-processamento
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif

# Bibliotecas para balanceamento de classes
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek

# Configurações gerais
warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("Bibliotecas importadas com sucesso")

## 2. Carregamento dos Dados e Insights da EDA

Carregamento dos datasets de treino e teste, seguido da documentação dos principais insights obtidos na análise exploratória que guiarão as decisões de preparação.

### 2.1 Carregamento dos Datasets

In [ ]:
train_df = pd.read_csv('dataset_sepsis_train.csv')

# Separar features e target
X_train = train_df.drop('SepsisLabel', axis=1)
y_train = train_df['SepsisLabel']

# Forma final
print(f"X_train: {X_train.shape} | y_train: {y_train.shape}")

# Distribuição das classes no treino
print(y_train.value_counts(normalize=True))

## 3. TAREFA 1: Seleção dos Dados

**Objetivo:** Escolher as variáveis mais relevantes para o modelo de mineração, removendo features com baixo potencial preditivo ou problemas graves de qualidade.

**Critérios de seleção:**
* Relevância clínica para detecção de sepsis
* Percentual de missing values aceitável
* Separabilidade entre classes (baseada na EDA)

### 3.1 Mapeamento de Variáveis com Excesso de Missing Values para Remoção

Vamos refazer a análise, mais objetiva e breve, das variáveis com >60% missing values para decidir quais manter, tratar ou remover baseado no critério de separabilidade de classes

In [ ]:

# Identificar variáveis com >60% missing
high_missing_vars = []
for col in X_train.select_dtypes(include=[np.number]).columns:
    missing_pct = (X_train[col].isnull().sum() / len(X_train)) * 100
    if missing_pct > 60:
        high_missing_vars.append({
            'variavel': col,
            'missing_pct': missing_pct
        })

# Calcular separabilidade para cada variável
separability_results = {
    'IMPUTAR_SIMPLES': [],     # Separabilidade > 0.3: Alta discriminação
    'IMPUTAR_AVANCADA': [],    # Separabilidade 0.16 - 0.3: Discriminação moderada  
    'DESCARTAR': []            # Separabilidade < 0.16: Baixa discriminação
}

for var_info in high_missing_vars:
    col = var_info['variavel']
    missing_pct = var_info['missing_pct']
    
    # Criar DataFrame temporário sem valores faltantes
    temp_df = pd.DataFrame({
        'feature': X_train[col],
        'target': y_train
    }).dropna()

    # Separar por classe
    no_sepsis_data = temp_df[temp_df['target'] == 0]['feature']
    sepsis_data = temp_df[temp_df['target'] == 1]['feature']

    # Calcular separabilidade (diferença de medianas / desvio padrão)
    median_diff = abs(sepsis_data.median() - no_sepsis_data.median())
    pooled_std = no_sepsis_data.std() if no_sepsis_data.std() > 0 else 1
    separability = median_diff / pooled_std
    
    # Classificar baseado na separabilidade
    var_result = {
        'variavel': col,
        'missing_pct': missing_pct,
        'separabilidade': separability,
        'n_amostras': len(temp_df)
    }
    
    if separability > 0.3:
        separability_results['IMPUTAR_SIMPLES'].append(var_result)
    elif separability >= 0.16:
        separability_results['IMPUTAR_AVANCADA'].append(var_result)
    else:
        separability_results['DESCARTAR'].append(var_result)

# Exibir resultados 

for categoria, vars_list in separability_results.items():
    print(f"\n{categoria} ({len(vars_list)} variáveis):")
    for var in sorted(vars_list, key=lambda x: x['separabilidade'], reverse=True):
        print(f"  • {var['variavel']}: Sep={var['separabilidade']:.3f} | Missing={var['missing_pct']:.1f}% | n={var['n_amostras']:,}")

variables_to_keep = [var['variavel'] for var in separability_results['IMPUTAR_SIMPLES']]
variables_to_treat = [var['variavel'] for var in separability_results['IMPUTAR_AVANCADA']]  
variables_to_discard = [var['variavel'] for var in separability_results['DESCARTAR']]

### 3.2 Análise de Separabilidade Estatística

Avaliação da capacidade discriminativa das variáveis que não foram selecionadas para exclusão, a fim de confirmar e justificar as decisões antes de fazer a remoção, usando testes estatísticos e métricas de separação entre classes.

In [ ]:
from scipy import stats
from sklearn.metrics import mutual_info_score

X_train_not_discard = X_train.drop(columns=variables_to_discard)
# Separar variáveis numéricas e categóricas
categorical_vars = ['Gender', 'Unit1', 'Unit2']  
# Numéricas são todas as colunas MENOS as categóricas
numeric_vars = [col for col in X_train_not_discard.columns if col not in categorical_vars]


# Análise para variáveis numéricas
all_separability_results = []

print(f"\nANÁLISE DE VARIÁVEIS NUMÉRICAS:")
print(f"{'Variável':<15} {'Missing%':<10} {'Separab.':<10} {'p-value_MW':<12} {'Mutual Info':<12} {'N_samples':<10}\n")

for var in numeric_vars:
    missing_pct = (X_train_not_discard[var].isnull().sum() / len(X_train_not_discard)) * 100
    
    # Criar DataFrame temporário sem valores faltantes
    temp_df = pd.DataFrame({
        'feature': X_train_not_discard[var],
        'target': y_train
    }).dropna()
    
    # Separar por classe
    no_sepsis_data = temp_df[temp_df['target'] == 0]['feature']
    sepsis_data = temp_df[temp_df['target'] == 1]['feature']

    # Calcular separabilidade (diferença de medianas / desvio padrão)
    median_diff = abs(sepsis_data.median() - no_sepsis_data.median())
    pooled_std = np.sqrt(((no_sepsis_data.std()**2 + sepsis_data.std()**2) / 2))
    separability = median_diff / pooled_std if pooled_std > 0 else 0
    
    # Teste U de Mann-Whitney (não-paramétrico)
    try:
        stat, p_value = stats.mannwhitneyu(sepsis_data, no_sepsis_data, alternative='two-sided')
        mann_whitney_pval = p_value
    except:
        mann_whitney_pval = 1.0  # p-value máximo para casos de erro

    # Informação mútua
    try:
        # Discretizar para mutual info (usar quintis)
        temp_df['feature_disc'] = pd.qcut(temp_df['feature'], q=5, labels=False, duplicates='drop')
        mutual_info = mutual_info_score(temp_df['target'], temp_df['feature_disc'])
    except:
        mutual_info = 0
    
    # Armazenar resultados
    result = {
        'variavel': var,
        'missing_pct': missing_pct,
        'separabilidade': separability,
        'mann_whitney': mann_whitney_pval,
        'mutual_info': mutual_info,
        'n_amostras': len(temp_df)
    }
    all_separability_results.append(result)
    
    # Exibir resultado
    print(f"{var:<15} {missing_pct:<10.1f} {separability:<10.3f} {mann_whitney_pval:<12.5f} {mutual_info:<12.7f} {len(temp_df):<10,}")


# Análise para variáveis categóricas
print(f"\nANÁLISE DE VARIÁVEIS CATEGÓRICAS:")
print(f"{'Variável':<15} {'Missing%':<10} {'p-value_Chi2':<12} {'Mutual Info':<12} {'N_samples':<10}\n")

for var in categorical_vars:
    missing_pct = (X_train_not_discard[var].isnull().sum() / len(X_train_not_discard)) * 100
    
    # Criar DataFrame temporário sem valores faltantes
    temp_df = pd.DataFrame({
        'feature': X_train_not_discard[var],
        'target': y_train
    }).dropna()
    
    # Teste Qui-quadrado
    try:
        contingency_table = pd.crosstab(temp_df['feature'], temp_df['target'])
        chi2, p_value, dof, expected = stats.chi2_contingency(contingency_table)
        chi2_pval = p_value
    except:
        chi2_pval = 1.0  # p-value máximo para casos de erro
    
    # Informação mútua
    try:
        mutual_info = mutual_info_score(temp_df['target'], temp_df['feature'])
    except:
        mutual_info = 0
    
    result = {
        'variavel': var,
        'missing_pct': missing_pct,
        'chi2_sig': chi2_pval,
        'mutual_info': mutual_info,
        'n_amostras': len(temp_df),
        'tipo': 'categorical'
    }
    all_separability_results.append(result)
    
    print(f"{var:<15} {missing_pct:<10.1f} {chi2_pval:<12.5f} {mutual_info:<12.7f} {len(temp_df):<10,}")


# Ranking por separabilidade (variáveis numéricas)
numeric_results = [r for r in all_separability_results if 'separabilidade' in r]
numeric_results_sorted = sorted(numeric_results, key=lambda x: x['separabilidade'], reverse=True)

print(f"\nSeparabilidades Numéricas:")
separabilities = [r['separabilidade'] for r in numeric_results]
print(f"  • Variáveis com Sep > 0.3: {sum(1 for s in separabilities if s > 0.3)}")
print(f"  • Variáveis com Sep > 0.16: {sum(1 for s in separabilities if s > 0.16)}")

# Análise de significância estatística
print(f"\nSIGNIFICÂNCIA ESTATÍSTICA (Numéricas):")
sig_001 = sum(1 for r in numeric_results if r['mann_whitney'] < 0.001)
sig_01 = sum(1 for r in numeric_results if 0.001 <= r['mann_whitney'] < 0.01)
sig_05 = sum(1 for r in numeric_results if 0.01 <= r['mann_whitney'] < 0.05)
not_sig = sum(1 for r in numeric_results if r['mann_whitney'] >= 0.05)
print(f"  • p < 0.001 (altamente significativo): {sig_001} variáveis")
print(f"  • 0.001 ≤ p < 0.01 (muito significativo): {sig_01} variáveis")  
print(f"  • 0.01 ≤ p < 0.05 (significativo): {sig_05} variáveis")
print(f"  • p ≥ 0.05 (não significativo): {not_sig} variáveis")

# Salvar resultados para uso posterior
statistical_analysis_results = {
    'numeric_results': numeric_results_sorted,
    'all_results': all_separability_results
}

#### Alteração Após Análise 
Percebe-se que ainda é possível remover `Age` do escopo de features visto que não há nenhuma métrica que aponte essa variável como algo relevante apesar do que diz a literatura sobre sepsis e o baixo percentual de missing values. Ela possui baixa separabilidade, um Man Whitney não significativo e Mutual Info demonstra zero informação sobre sepse

| Variável | Missing% | Separabilidade |  p-value_MW | Mutual Info |
|----------|----------|----------------|--------------|-------------| 
| **Age** | 0.0 | 0.000 |  0.4367935 | 0.00000 |


### 3.3 Aplicação das Decisões de Separabilidade

Implementação prática da remoção de variáveis com baixa separabilidade e organização das listas para tratamento adiante.

In [ ]:
# Adicionar Age às variáveis a descartar 
variables_to_discard.append('Age')

# Remover variáveis com baixa separabilidade e alto missing do dataset principal
total_to_remove = len(variables_to_discard)
print(f"Removendo {total_to_remove} variáveis com baixa separabilidade...")

X_train_selected = X_train.drop(columns=variables_to_discard)

print("Variáveis removidas:")
for var in variables_to_discard:
    missing_pct = (X_train[var].isnull().sum() / len(X_train)) * 100
    if var == 'Age':
        print(f"  • {var}: {missing_pct:.1f}% missing (removida por baixa discriminação)")
    else:
        print(f"  • {var}: {missing_pct:.1f}% missing")

print(f"\nDimensões do dataset:")
print(f"  • Original: {X_train.shape}")
print(f"  • Após seleção: {X_train_selected.shape}")

# Organizar variáveis por estratégia de tratamento
high_missing_strategy = {
    'imputacao_simples': variables_to_keep,      
    'imputacao_avancada': variables_to_treat,    
    'removidas': variables_to_discard           
}


### 3.4 Síntese das Decisões de Seleção de Variáveis

**Documentação completa das decisões tomadas na Tarefa 1 (Seleção dos Dados) com respectivas justificativas:**


RESULTADOS FINAIS DA SELEÇÃO DE VARIÁVEIS

CRITÉRIOS DE SELEÇÃO APLICADOS

1. **Critério de Missing Values**: Variáveis com >60% de valores ausentes analisadas
2. **Critério de Separabilidade**: Capacidade discriminativa entre classes (limite: 0.16)
3. **Critério Estatístico**: Significância nos testes Mann-Whitney U e Chi-quadrado

IMPACTO FINAL DAS DECISÕES

**Redução Dimensional Efetiva:**
- **Dataset original**: 1,241,768 × 41 variáveis
- **Dataset final**: 1,241,768 × 16 variáveis  
- **Redução**: 61% das variáveis removidas (25/41)
- **Taxa de compressão**: 2.6:1

ESTRATÉGIAS DE TRATAMENTO DEFINIDAS

**IMPUTAÇÃO Cuidadosa** (1 variáveis - Separabilidade > 0.3)

**Estratégia**: Imputação com medidas robustas (mediana) + validação clínica

| Variável | Missing% | Separabilidade | p-value | Justificativa Médica |
|----------|----------|----------------|---------|---------------------|
| **Temp** | 66.2% | 0.326 | < 0.001 | Temperatura corporal: indicador direto de resposta inflamatória |

**IMPUTAÇÃO Específica** (3 variáveis - Separabilidade 0.16-0.3 ou Separabilidade>0.3 e Missing>90%)  

**Estratégia**: Técnicas sofisticadas (KNN, regressão) devido à considerável relevância clínica 

| Variável | Missing% | Separabilidade | p-value | Justificativa Médica |
|----------|----------|----------------|---------|---------------------|
| **BUN** | 93.1% | 0.328 | < 0.001 | Função renal: biomarcador de disfunção orgânica na sepsis |
| **Platelets** | 94.1% | 0.189 | < 0.001 | Coagulação: trombocitopenia marca disfunção hemostática |
| **WBC** | 93.6% | 0.166 | < 0.001 | Sistema imune: resposta leucocitária à infecção |

**REMOVIDAS** (25 variáveis - Separabilidade < 0.16)

**Critério duplo**: Baixa discriminação + Alto missing (>60%)

**Destaques das remoções:**
- **Age**: 0.0% missing, Sep: 0.000, p-value: 0.437 (única exceção por baixa discriminação)
- **24 variáveis** com >80% missing + separabilidade < 0.16
- **Maior redução**: TroponinI (99.1% missing), Bilirubin_direct (99.8% missing)

VALIDAÇÃO ESTATÍSTICA FINAL

**Testes Aplicados:**
- **Mann-Whitney U**: Para variáveis numéricas (não-paramétrico)
- **Qui-quadrado**: Para variáveis categóricas
- **Informação Mútua**: Medida de dependência entre variáveis

**Significância dos Testes:**
- **Variáveis numéricas significativas**: 13/14 (p < 0.05)
- **Variáveis categóricas significativas**: 3/3 (p < 0.001)  
- **Taxa de significância geral**: 94.1% (16/17 variáveis)

**Estratégias de Tratamento Definidas:**
- **Imputação cuidadosa**: 2 variáveis de alta relevância
- **Imputação específica**: 6 variáveis de relevância moderada
- **Manutenção**: 13 variáveis com baixo missing

## 4. TAREFA 2: Limpeza dos Dados

**Objetivo:** Corrigir ou remover dados inconsistentes, duplicados ou ausentes através de estratégias específicas para cada tipo de variável.

**Estratégias por tipo de missing:**
* Missing < 20%: Imputação simples (mediana/moda)
* Missing >= 20%: Imputação baseada em modelos

### 4.1 Detecção e Remoção de Duplicatas

Identificação de registros duplicados exatos e tratamento adequado considerando a natureza temporal dos dados.

In [54]:
# Verificar duplicatas no dataset selecionado
print("DETECÇÃO DE DUPLICATAS:")
print(f"Dataset atual: {X_train_selected.shape}")

# Verificar duplicatas exatas (todas as colunas)
duplicatas_exatas = X_train_selected.duplicated().sum()
print(f"Duplicatas exatas encontradas: {duplicatas_exatas:,}")

print(f"Remover {duplicatas_exatas:,} duplicatas exatas")
X_train_cleaned = X_train_selected.drop_duplicates()
y_train_cleaned = y_train.loc[X_train_cleaned.index]
print(f"Dataset após remoção: {X_train_cleaned.shape}")

print(f"Dataset limpo final: {X_train_cleaned.shape}")

DETECÇÃO DE DUPLICATAS:
Dataset atual: (1241768, 16)
Duplicatas exatas encontradas: 32,571
Remover 32,571 duplicatas exatas
Dataset após remoção: (1209197, 16)
Dataset limpo final: (1209197, 16)


### 4.2 Tratamento e Análise de Outliers

A ideia é tentar preservar os outliers visto que eles se demonstraram relevantes para a identificação de instâncias com SepsisLabel=1 na Análise Exploratória.
Vamos apenas deixar algumas variáveis mais genéricas e conhecidas mais consistentes e fazer uma análise geral.

In [68]:
# Tratamento de outliers para variáveis numéricas
from scipy import stats

print("DETECÇÃO E TRATAMENTO DE OUTLIERS:\n")

# Separar variáveis numéricas do dataset limpo
numeric_cols = X_train_cleaned.select_dtypes(include=[np.number]).columns.tolist()
print(f"Variáveis numéricas para análise: {len(numeric_cols)}")

# Definir limites um pouco mais realistas para algumas variáveis 
# Considerando registros de outros casos extremos e do próprio dataset
# Propósito de deixar os dados mais consistentes
clinical_limits = {
    'HR': (20, 250),           # Batimentos cardíacos: 20-250 bpm
    'Temp': (28, 42),          # Temperatura: 28-42°C
    'Hour': (0, 336),          # Horas na UTI: 1-336h (14 dias)
    'ICULOS': (0, 336),        # Tempo UTI: 1-336h
    'HospAdmTime': (0, 24),   # Tempo hospital: 0 a 24h
}

outliers_summary = {}
X_train_outliers_treated = X_train_cleaned.copy()

for col in numeric_cols:
    data = X_train_outliers_treated[col].dropna()
        
    # Cap do Range (quando aplicável)
    clinical_outliers = 0
    if col in clinical_limits:
        min_val, max_val = clinical_limits[col]
        clinical_mask = (data < min_val) | (data > max_val)
        clinical_outliers = clinical_mask.sum()
        
        # Aplicar capping
        X_train_outliers_treated.loc[X_train_outliers_treated[col] < min_val, col] = min_val
        X_train_outliers_treated.loc[X_train_outliers_treated[col] > max_val, col] = max_val
    
    # Análise do IQR
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    iqr_outliers = ((data < lower_bound) | (data > upper_bound)).sum()
    
    # Análise do Z-score (outliers > 3 desvios padrão)
    z_scores = np.abs(stats.zscore(data))
    zscore_outliers = (z_scores > 3).sum()
    
    outliers_summary[col] = {
        'clinical': clinical_outliers,
        'iqr': iqr_outliers,
        'zscore': zscore_outliers,
        'total_values': len(data),
        'range_original': (data.min(), data.max()),
        'range_treated': (X_train_outliers_treated[col].min(), X_train_outliers_treated[col].max())
    }

# Exibir resumo dos outliers
print(f"\nRESUMO DE OUTLIERS DETECTADOS:")
print(f"{'Variável':<12} {'Clínicos':<9} {'IQR':<8} {'Z-score':<8} {'N_total':<8} {'Range Original':<20} {'Range Tratado':<20}")
print("-" * 100)

for col, summary in outliers_summary.items():
    clinical = summary['clinical']
    iqr = summary['iqr'] 
    zscore = summary['zscore']
    total = summary['total_values']
    range_orig = f"{summary['range_original'][0]:.2f}-{summary['range_original'][1]:.2f}"
    range_treat = f"{summary['range_treated'][0]:.2f}-{summary['range_treated'][1]:.2f}"

    print(f"{col:<12} {clinical:<9,} {iqr:<8,} {zscore:<8,} {total:<8,} {range_orig:<20} {range_treat:<20}")

# Estatísticas finais
total_clinical_corrections = sum(summary['clinical'] for summary in outliers_summary.values())
print(f"\nTotal de Caps Aplicados: {total_clinical_corrections:,}")

DETECÇÃO E TRATAMENTO DE OUTLIERS:

Variáveis numéricas para análise: 16

RESUMO DE OUTLIERS DETECTADOS:
Variável     Clínicos  IQR      Z-score  N_total  Range Original       Range Tratado       
----------------------------------------------------------------------------------------------------
Hour         0         54,951   27,883   1,209,197 0.00-335.00          0.00-335.00         
HR           2         11,203   5,502    1,119,122 20.00-280.00         20.00-250.00        
O2Sat        0         19,905   8,912    1,079,707 20.00-100.00         20.00-100.00        
Temp         11        5,223    3,392    419,945  23.00-50.00          28.00-42.00         
SBP          0         12,748   6,046    1,060,857 20.00-300.00         20.00-300.00        
MAP          0         17,543   8,057    1,087,236 20.00-300.00         20.00-300.00        
DBP          0         13,033   6,560    852,691  20.00-300.00         20.00-300.00        
Resp         0         22,208   10,343   1,051,178 1.

### 4.3 Estratégias de Imputação

Implementação de diferentes técnicas de imputação baseadas no tipo de variável e percentual de missing values.

**OBSERVAÇÃO:**
As seções `4.3.1` e `4.3.2` precisam ser executadas em ordem e são necessárias para que as demais seções funcionem. Porém `4.3.3`, `4.3.4`, `4.3.5` podem ser executadas em qualquer ordem após executar `4.3.1` e `4.3.2`

#### 4.3.1 Imputação para Variáveis com Baixo Missing (<20%)

Aplicação de imputação simples usando medidas centrais apropriadas para cada tipo de variável.

In [69]:
# Imputação simples para variáveis com baixo missing (<20%)
from sklearn.impute import SimpleImputer


X_train_simple_imputed = X_train_outliers_treated.copy()

# Identificar variáveis com baixo missing (<20%)
low_missing_vars = []
missing_info = {}

for col in X_train_simple_imputed.columns:
    missing_pct = (X_train_simple_imputed[col].isnull().sum() / len(X_train_simple_imputed)) * 100
    missing_info[col] = missing_pct
    
    if missing_pct < 20 and missing_pct > 0:
        low_missing_vars.append(col)


print(f"\nVariáveis para imputação simples:")
for var in low_missing_vars:
    missing_count = X_train_simple_imputed[var].isnull().sum()
    missing_pct = missing_info[var]
    print(f"  • {var}: {missing_count:,} valores ({missing_pct:.1f}%)")


numeric_imputer = SimpleImputer(strategy='median')
X_train_simple_imputed[low_missing_vars] = numeric_imputer.fit_transform(
    X_train_simple_imputed[low_missing_vars]
)


# Verificar se imputação foi bem-sucedida
print(f"\nVERIFICAÇÃO PÓS-IMPUTAÇÃO:")
for var in low_missing_vars:
    remaining_missing = X_train_simple_imputed[var].isnull().sum()
    print(f"  • {var}: {remaining_missing} valores missing restantes")

total_imputed = sum(missing_info[var] * len(X_train_simple_imputed) / 100 for var in low_missing_vars)
print(f"\nTotal de valores imputados (simples): {total_imputed:,.0f}")

# Resumo do missing restante
remaining_missing = X_train_simple_imputed.isnull().sum().sum()
total_values = X_train_simple_imputed.size
missing_pct_remaining = (remaining_missing / total_values) * 100

print(f"Missing values restantes: {remaining_missing:,} ({missing_pct_remaining:.2f}%)")



Variáveis para imputação simples:
  • HR: 90,075 valores (7.4%)
  • O2Sat: 129,490 valores (10.7%)
  • SBP: 148,340 valores (12.3%)
  • MAP: 121,961 valores (10.1%)
  • Resp: 158,019 valores (13.1%)
  • HospAdmTime: 6 valores (0.0%)

VERIFICAÇÃO PÓS-IMPUTAÇÃO:
  • HR: 0 valores missing restantes
  • O2Sat: 0 valores missing restantes
  • SBP: 0 valores missing restantes
  • MAP: 0 valores missing restantes
  • Resp: 0 valores missing restantes
  • HospAdmTime: 0 valores missing restantes

Total de valores imputados (simples): 647,891
Missing values restantes: 5,478,673 (28.32%)


#### 4.3.2 Separando as Demais Variáveis para Imputação Avançada 

Para uso de técnicas mais sofisticadas como KNN Imputer ou imputação baseada em modelos para variáveis clinicamente importantes, vamos antes definir as variáveis que ainda possuem valores faltantes

In [70]:

X_train_advanced_imputed = X_train_simple_imputed.copy()

# ESTRATÉGIA DINÂMICA BASEADA EM MISSING ATUAL
print(f"\nANÁLISE DINÂMICA DE MISSING VALUES:")
print(f"="*50)

# Identificar todas as variáveis com missing
vars_with_missing = []
for col in X_train_advanced_imputed.columns:
    missing_count = X_train_advanced_imputed[col].isnull().sum()
    missing_pct = (missing_count / len(X_train_advanced_imputed)) * 100
    
    if missing_count > 0:
        vars_with_missing.append({
            'variavel': col,
            'missing_count': missing_count,
            'missing_pct': missing_pct,
            'tipo': 'categorical' if X_train_advanced_imputed[col].dtype in ['object', 'category'] else 'numeric'
        })

# Separar por estratégia baseada em 40% de missing e tipo de variável
logistic_regression_vars = []  # Unit1, Unit2: Regressão Logística (categóricas)
linear_regression_vars = []    # < 40% missing: Regressão Linear simples (numéricas)
hybrid_strategy_vars = []      # >= 40% missing: Estratégia Híbrida KNN + Regressão

print(f"\nCLASSIFICAÇÃO POR ESTRATÉGIA DE IMPUTAÇÃO:")
print(f"Critério: Unit1/Unit2 = Regressão Logística | <40% = Regressão Linear | ≥40% = Híbrida")
print(f"-" * 85)

for var_info in sorted(vars_with_missing, key=lambda x: x['missing_pct']):
    var = var_info['variavel']
    missing_pct = var_info['missing_pct']
    missing_count = var_info['missing_count']
    
    print(f"{var:<15} {missing_pct:<8.1f}% ({missing_count:>8,} valores)")
    
    # Separar Unit1 e Unit2 para regressão logística
    if var in ['Unit1', 'Unit2']:
        logistic_regression_vars.append(var)
    elif missing_pct < 40:
        linear_regression_vars.append(var)
    else:
        hybrid_strategy_vars.append(var)

print(f"\nRESUMO DAS ESTRATÉGIAS:")
print(f"  • Regressão Logística (categóricas Unit1/Unit2): {len(logistic_regression_vars)} variáveis")
print(f"  • Regressão Linear Simples (<40% missing numéricas): {len(linear_regression_vars)} variáveis")
print(f"  • Estratégia Híbrida (≥40% missing numéricas): {len(hybrid_strategy_vars)} variáveis")

print(f"\nVariáveis para Regressão Logística: {logistic_regression_vars}")
print(f"Variáveis para Regressão Linear: {linear_regression_vars}")
print(f"Variáveis para Estratégia Híbrida: {hybrid_strategy_vars}")

# Selecionar variáveis preditoras (sem missing ou baixo missing)
all_vars_to_impute = logistic_regression_vars + linear_regression_vars + hybrid_strategy_vars
predictor_vars = []
for col in X_train_advanced_imputed.select_dtypes(include=[np.number]).columns:
    missing_pct = (X_train_advanced_imputed[col].isnull().sum() / len(X_train_advanced_imputed)) * 100
    if missing_pct == 0 or (missing_pct < 20 and col not in all_vars_to_impute):
        predictor_vars.append(col)

print(f"\nVariáveis preditoras selecionadas: {predictor_vars}")




ANÁLISE DINÂMICA DE MISSING VALUES:

CLASSIFICAÇÃO POR ESTRATÉGIA DE IMPUTAÇÃO:
Critério: Unit1/Unit2 = Regressão Logística | <40% = Regressão Linear | ≥40% = Híbrida
-------------------------------------------------------------------------------------
DBP             29.5    % ( 356,506 valores)
Unit1           39.0    % ( 472,083 valores)
Unit2           39.0    % ( 472,083 valores)
Temp            65.3    % ( 789,252 valores)
BUN             92.9    % (1,123,758 valores)
WBC             93.4    % (1,129,584 valores)
Platelets       93.9    % (1,135,407 valores)

RESUMO DAS ESTRATÉGIAS:
  • Regressão Logística (categóricas Unit1/Unit2): 2 variáveis
  • Regressão Linear Simples (<40% missing numéricas): 1 variáveis
  • Estratégia Híbrida (≥40% missing numéricas): 4 variáveis

Variáveis para Regressão Logística: ['Unit1', 'Unit2']
Variáveis para Regressão Linear: ['DBP']
Variáveis para Estratégia Híbrida: ['Temp', 'BUN', 'WBC', 'Platelets']

Variáveis preditoras selecionadas: ['Hour',

#### 4.3.3 Regressão Logística para Categóricas

Aqui aplicaremos o modelo de regressão logistica para imputação de Unit1 e Unit2. Mantendo a coerência do Dataset onde é mandatório que Unit1 + Unit2 = 1 para todas as instâncias (Paciente só pode ir para um dos tipos de UTI) 

In [71]:
# =============================================================================
# ETAPA 1: REGRESSÃO LOGÍSTICA PARA VARIÁVEIS CATEGÓRICAS (Unit1, Unit2)
# =============================================================================

from sklearn.linear_model import LogisticRegression

# Identificar valores missing
unit1_missing_mask = X_train_advanced_imputed['Unit1'].isnull()
unit2_missing_mask = X_train_advanced_imputed['Unit2'].isnull()
both_missing_mask = unit1_missing_mask & unit2_missing_mask

unit1_missing_count = unit1_missing_mask.sum()
unit2_missing_count = unit2_missing_mask.sum()
both_missing_count = both_missing_mask.sum()

print(f"Unit1 missing: {unit1_missing_count:,} valores")
print(f"Unit2 missing: {unit2_missing_count:,} valores") 
print(f"Ambas missing: {both_missing_count:,} valores")

print(f"\nETAPA 1: Regressão Logística para Unit1")

# Preparar dados para treino (onde Unit1 não é missing)
complete_mask = ~X_train_advanced_imputed['Unit1'].isnull()
training_size = min(100000, complete_mask.sum())

training_indices = np.random.choice(
    X_train_advanced_imputed[complete_mask].index,
    size=training_size,
    replace=False
)

# Features numéricas para predição (excluir Unit1/Unit2)
numeric_predictors = [col for col in predictor_vars if col not in logistic_regression_vars]

# Treinar modelo logístico para Unit1
X_features = X_train_advanced_imputed.loc[training_indices, numeric_predictors]
y_target = X_train_advanced_imputed.loc[training_indices, 'Unit1']

logistic_model = LogisticRegression(random_state=42, max_iter=1000)
logistic_model.fit(X_features, y_target)

# Prever Unit1 para registros missing
X_missing_features = X_train_advanced_imputed.loc[both_missing_mask, numeric_predictors]
predicted_unit1_proba = logistic_model.predict_proba(X_missing_features)[:, 1]
predicted_unit1 = (predicted_unit1_proba > 0.5).astype(int)

print(f"ETAPA 2: Unit2 como complemento de Unit1")
# Unit2 como complemento lógico de Unit1
predicted_unit2 = 1 - predicted_unit1

# Aplicar imputações
X_train_advanced_imputed.loc[both_missing_mask, 'Unit1'] = predicted_unit1
X_train_advanced_imputed.loc[both_missing_mask, 'Unit2'] = predicted_unit2

print(f"Unit1 imputado: {both_missing_count:,} valores")
print(f"Unit2 imputado: {both_missing_count:,} valores (complemento)")

# Estatísticas da imputação
unit1_0_count = (predicted_unit1 == 0).sum()
unit1_1_count = (predicted_unit1 == 1).sum()
print(f"Distribuição Unit1 imputada: 0={unit1_0_count:,} | 1={unit1_1_count:,}")
print(f"Distribuição Unit2 imputada: 0={unit1_1_count:,} | 1={unit1_0_count:,}")

# Verificar relação complementar
unit1_final = X_train_advanced_imputed.loc[both_missing_mask, 'Unit1']
unit2_final = X_train_advanced_imputed.loc[both_missing_mask, 'Unit2']
sum_check = (unit1_final + unit2_final == 1).all()
print(f"Verificação Unit1 + Unit2 = 1: {'✓ VÁLIDA' if sum_check else '✗ INVÁLIDA'}")
    
# Verificação pós-imputação logística
print(f"\nVERIFICAÇÃO PÓS-IMPUTAÇÃO LOGÍSTICA:")
for var in logistic_regression_vars:
    if var in X_train_advanced_imputed.columns:
        remaining_missing = X_train_advanced_imputed[var].isnull().sum()
        print(f"  • {var}: {remaining_missing} valores missing restantes")
        
        # Distribuição final
        if remaining_missing == 0:
            value_counts = X_train_advanced_imputed[var].value_counts().sort_index()
            print(f"    Distribuição final: {dict(value_counts)}")

Unit1 missing: 472,083 valores
Unit2 missing: 472,083 valores
Ambas missing: 472,083 valores

ETAPA 1: Regressão Logística para Unit1
ETAPA 2: Unit2 como complemento de Unit1
Unit1 imputado: 472,083 valores
Unit2 imputado: 472,083 valores (complemento)
Distribuição Unit1 imputada: 0=178,891 | 1=293,192
Distribuição Unit2 imputada: 0=293,192 | 1=178,891
Verificação Unit1 + Unit2 = 1: ✓ VÁLIDA

VERIFICAÇÃO PÓS-IMPUTAÇÃO LOGÍSTICA:
  • Unit1: 0 valores missing restantes
    Distribuição final: {0.0: np.int64(553649), 1.0: np.int64(655548)}
  • Unit2: 0 valores missing restantes
    Distribuição final: {0.0: np.int64(655548), 1.0: np.int64(553649)}


#### 4.3.4 Regressão Linear para <40% de Missing
Aplicar regressão linear para variáveis numéricas com <40% de missing (DBP)

In [ ]:
# =============================================================================
# ETAPA 2: REGRESSÃO LINEAR SIMPLES (< 40% missing)
# =============================================================================

from sklearn.linear_model import LinearRegression

print(f"ETAPA 2: REGRESSÃO LINEAR SIMPLES (< 40% missing)\n")

for target_var in linear_regression_vars:
    print(f"\n---------- IMPUTANDO {target_var} (Regressão Simples) ----------")
    
    # Identificar valores missing
    missing_mask = X_train_advanced_imputed[target_var].isnull()
    total_missing = missing_mask.sum()
    missing_pct = (total_missing / len(X_train_advanced_imputed)) * 100
    print(f"Missing: {total_missing:,} valores ({missing_pct:.1f}%)")

    # Preparar dados para regressão
    complete_mask = ~X_train_advanced_imputed[target_var].isnull()
    training_size = min(100000, complete_mask.sum())  # Até 100k para treino
    
    training_indices = np.random.choice(
        X_train_advanced_imputed[complete_mask].index,
        size=training_size,
        replace=False
    )
    
    # Features e target para treino
    X_features = X_train_advanced_imputed.loc[training_indices, predictor_vars]
    y_target = X_train_advanced_imputed.loc[training_indices, target_var]
    
    # Treinar modelo de regressão
    reg_model = LinearRegression()
    reg_model.fit(X_features, y_target)
    
    # Prever todos os valores missing
    X_missing_features = X_train_advanced_imputed.loc[missing_mask, predictor_vars]
    predicted_values = reg_model.predict(X_missing_features)
    
    # Aplicar imputação
    X_train_advanced_imputed.loc[missing_mask, target_var] = predicted_values
    
    # Verificar resultado
    final_missing = X_train_advanced_imputed[target_var].isnull().sum()
    print(f"Regressão aplicada: {total_missing:,} valores imputados")
    print(f"Valores restantes missing: {final_missing}")
    print(f"Range imputado: [{predicted_values.min():.2f}, {predicted_values.max():.2f}]")


ETAPA 2: REGRESSÃO LINEAR SIMPLES (< 40% missing)


---------- IMPUTANDO DBP (Regressão Simples) ----------
Missing: 356,506 valores (29.5%)
Regressão aplicada: 356,506 valores imputados
Valores restantes missing: 0
Range imputado: [2.99, 259.58]
Range atual da variável: [2.99, 300.00]


#### 4.3.5 KNNImputer + Regressão Linear para >=40% de Missing
Aplicação do KNNImputer com 3 vizinhos para amostra de ~5% dos valores da coluna e Regressão Linear para os outros ~95%.

Esse código abaixo demora em torno de 5-6min para executar. Tomar ciência disso antes rodar a célula de código abaixo 

In [ ]:
# =============================================================================
# ETAPA 3: ESTRATÉGIA HÍBRIDA (≥ 40% missing)  
# =============================================================================

# Imputação avançada híbrida: KNN + Regressão Linear
from sklearn.impute import KNNImputer

# Implementar estratégia híbrida para cada variável
for target_var in hybrid_strategy_vars:
    print(f"\n========== IMPUTANDO {target_var} ==========")
    
    # Identificar valores missing
    missing_mask = X_train_advanced_imputed[target_var].isnull()
    total_missing = missing_mask.sum()
    print(f"Total de valores missing: {total_missing:,}")
    
    # ETAPA 1: Imputação com KNN (5% dos missing values - otimizado)
    knn_sample_size = min(50000, int(total_missing * 0.05))  # Máximo 50k valores
    print(f"\nETAPA 1 - KNN Imputer ({knn_sample_size:,} valores - 5%)")
    
    # Selecionar amostra aleatória dos índices missing para KNN
    missing_indices = X_train_advanced_imputed[missing_mask].index
    knn_indices = np.random.choice(missing_indices, size=knn_sample_size, replace=False)
    
    # Preparar subset para KNN (incluir valores não-missing para treino)
    mask_not_missing = ~X_train_advanced_imputed[target_var].isnull()
    knn_training_size = min(10000, mask_not_missing.sum())  # Reduzir para 10k treino
    training_indices = np.random.choice(
        X_train_advanced_imputed[mask_not_missing].index, 
        size=knn_training_size, 
        replace=False
    )
    
    # Combinar índices para KNN: treino + amostra para imputação
    knn_all_indices = np.concatenate([training_indices, knn_indices])
    knn_subset = X_train_advanced_imputed.loc[knn_all_indices, predictor_vars + [target_var]].copy()
    
    # Aplicar KNN apenas no subset
    knn_imputer = KNNImputer(n_neighbors=3, weights='uniform')
    knn_imputed = knn_imputer.fit_transform(knn_subset)
    
    # Extrair valores imputados para a variável alvo
    target_col_idx = list(knn_subset.columns).index(target_var)
    knn_imputed_values = knn_imputed[-knn_sample_size:, target_col_idx]
    
    # Atualizar valores no dataset
    X_train_advanced_imputed.loc[knn_indices, target_var] = knn_imputed_values
    print(f"  KNN aplicado com sucesso: {knn_sample_size:,} valores")
    
    # ETAPA 2: Imputação com Regressão Linear (95% restante)
    remaining_missing_mask = X_train_advanced_imputed[target_var].isnull()
    remaining_missing_count = remaining_missing_mask.sum()
    print(f"\nETAPA 2 - Regressão Linear ({remaining_missing_count:,} valores - 95%)")
    
    # Preparar dados para regressão (usar todos os dados completos)
    complete_mask = ~X_train_advanced_imputed[target_var].isnull()
    reg_training_size = min(50000, complete_mask.sum())
    
    reg_training_indices = np.random.choice(
        X_train_advanced_imputed[complete_mask].index,
        size=reg_training_size,
        replace=False
    )
    
    # Features e target para treino
    X_features = X_train_advanced_imputed.loc[reg_training_indices, predictor_vars]
    y_target = X_train_advanced_imputed.loc[reg_training_indices, target_var]
    
    # Treinar modelo de regressão
    reg_model = LinearRegression()
    reg_model.fit(X_features, y_target)
    
    # Prever valores restantes
    X_missing_features = X_train_advanced_imputed.loc[remaining_missing_mask, predictor_vars]
    predicted_values = reg_model.predict(X_missing_features)
    
    X_train_advanced_imputed.loc[remaining_missing_mask, target_var] = predicted_values
    print(f"  Regressão aplicada com sucesso: {remaining_missing_count:,} valores")
            
    
    # Verificar se imputação foi completa
    final_missing = X_train_advanced_imputed[target_var].isnull().sum()
    print(f"  Valores missing restantes: {final_missing}")
    print(f"  Imputação híbrida completa para {target_var}!")

print("Imputação Concluída!")


========== IMPUTANDO Temp ==========
Total de valores missing: 789,252

ETAPA 1 - KNN Imputer (39,462 valores - 5%)
  KNN aplicado com sucesso: 39,462 valores

ETAPA 2 - Regressão Linear (749,790 valores - 95%)
  Regressão aplicada com sucesso: 749,790 valores
  Valores missing restantes: 0
  Imputação híbrida completa para Temp!

========== IMPUTANDO BUN ==========
Total de valores missing: 1,123,758

ETAPA 1 - KNN Imputer (50,000 valores - 5%)
  KNN aplicado com sucesso: 50,000 valores

ETAPA 2 - Regressão Linear (1,073,758 valores - 95%)
  Regressão aplicada com sucesso: 1,073,758 valores
  Valores missing restantes: 0
  Imputação híbrida completa para BUN!

========== IMPUTANDO WBC ==========
Total de valores missing: 1,129,584

ETAPA 1 - KNN Imputer (50,000 valores - 5%)
  KNN aplicado com sucesso: 50,000 valores

ETAPA 2 - Regressão Linear (1,079,584 valores - 95%)
  Regressão aplicada com sucesso: 1,079,584 valores
  Valores missing restantes: 0
  Imputação híbrida completa par

#### 4.3.6 Aplicando os Limites Pós-Imputação

Definimos o range de valores segundo a análise feita na seção 4.2 para não permitir que nenhum outlier imputado ultrapasse o intervalo observado

In [77]:
# =============================================================================
# APLICAÇÃO DE CAPS
# =============================================================================

print(f"\n" + "="*60)
print(f"APLICAÇÃO DE CAPS PÓS-IMPUTAÇÃO")
print(f"="*60)

# Definir ranges tratados do outliers_summary (Seção 4.2)
caps_ranges = {
    'Temp': {'min': 28.00, 'max': 42.00},
    'BUN': {'min': 1.00, 'max': 268.00},
    'WBC': {'min': 0.10, 'max': 440.00},
    'Platelets': {'min': 2.00, 'max': 2322.00},
    'DBP': {'min': 20.00, 'max': 300.00}
}

# Aplicar caps nas variáveis imputadas
total_caps_applied = 0

for var in caps_ranges.keys():
    if var in X_train_advanced_imputed.columns:
        min_cap = caps_ranges[var]['min']
        max_cap = caps_ranges[var]['max']
        
        # Contabilizar valores fora dos limites antes da correção
        below_min = (X_train_advanced_imputed[var] < min_cap).sum()
        above_max = (X_train_advanced_imputed[var] > max_cap).sum()
        total_corrections = below_min + above_max
        
        if total_corrections > 0:
            print(f"  • {var}: {below_min:,} valores < {min_cap}, {above_max:,} valores > {max_cap}")
            
            # Aplicar caps
            X_train_advanced_imputed[var] = X_train_advanced_imputed[var].clip(
                lower=min_cap, 
                upper=max_cap
            )
            
            total_caps_applied += total_corrections
        else:
            print(f"  • {var}: Nenhuma correção necessária")

print(f"\nTotal de caps aplicados: {total_caps_applied:,}")

# Verificar ranges após caps
print(f"\nRANGES APÓS APLICAÇÃO DE CAPS:")
for var in caps_ranges.keys():
    if var in X_train_advanced_imputed.columns:
        min_val = X_train_advanced_imputed[var].min()
        max_val = X_train_advanced_imputed[var].max()
        print(f"  • {var}: {min_val:.2f} - {max_val:.2f}")


APLICAÇÃO DE CAPS PÓS-IMPUTAÇÃO
  • Temp: Nenhuma correção necessária
  • BUN: Nenhuma correção necessária
  • WBC: Nenhuma correção necessária
  • Platelets: Nenhuma correção necessária
  • DBP: Nenhuma correção necessária

Total de caps aplicados: 0

RANGES APÓS APLICAÇÃO DE CAPS:
  • Temp: 28.00 - 42.00
  • BUN: 1.00 - 268.00
  • WBC: 0.10 - 440.00
  • Platelets: 2.00 - 2322.00
  • DBP: 20.00 - 300.00


#### 4.3.7 Verificação Final Pós-Imputação

In [ ]:
# =============================================================================
# VERIFICAÇÃO FINAL PÓS-IMPUTAÇÃO
# =============================================================================

print(f"\n" + "="*60)
print(f"VERIFICAÇÃO FINAL PÓS-IMPUTAÇÃO")
print(f"="*60)

# Verificar todas as variáveis processadas
all_processed_vars = linear_regression_vars + hybrid_strategy_vars + logistic_regression_vars

print(f"\nVALIDAÇÃO DE RANGES PÓS-IMPUTAÇÃO:")

# Variáveis processadas
numeric_processed = linear_regression_vars + hybrid_strategy_vars + logistic_regression_vars
for var in numeric_processed:
    if var in X_train_advanced_imputed.columns:
        min_val = X_train_advanced_imputed[var].min()
        max_val = X_train_advanced_imputed[var].max()
        mean_val = X_train_advanced_imputed[var].mean()
        print(f"  • {var}: min={min_val:.2f}, max={max_val:.2f}, mean={mean_val:.2f}")


# Verificar se ainda há variáveis com missing
print(f"\nVERIFICAÇÃO GERAL DE MISSING:")
total_vars_with_missing = 0
for col in X_train_advanced_imputed.columns:
    missing_count = X_train_advanced_imputed[col].isnull().sum()
    if missing_count > 0:
        missing_pct = (missing_count / len(X_train_advanced_imputed)) * 100
        print(f"  • {col}: {missing_count:,} valores ({missing_pct:.1f}%)")
        total_vars_with_missing += 1

if total_vars_with_missing == 0:
    print("NENHUMA VARIÁVEL COM MISSING RESTANTE!")


print(f"\nDataset após imputações: {X_train_advanced_imputed.shape}")

# Status final do missing
final_missing = X_train_advanced_imputed.isnull().sum().sum()
total_values = X_train_advanced_imputed.size
final_missing_pct = (final_missing / total_values) * 100

print(f"Missing values finais: {final_missing:,} ({final_missing_pct:.3f}%)")


VERIFICAÇÃO FINAL PÓS-IMPUTAÇÃO

VALIDAÇÃO DE RANGES PÓS-IMPUTAÇÃO:
  • DBP: min=20.00, max=300.00, mean=62.67
  • Temp: min=28.00, max=42.00, mean=36.96
  • BUN: min=1.00, max=268.00, mean=24.28
  • WBC: min=0.10, max=440.00, mean=11.37
  • Platelets: min=2.00, max=2322.00, mean=196.22
  • Unit1: min=0.00, max=1.00, mean=0.54
  • Unit2: min=0.00, max=1.00, mean=0.46

VERIFICAÇÃO GERAL DE MISSING:
  ✓ NENHUMA VARIÁVEL COM MISSING RESTANTE!

Dataset após imputação avançada: (1209197, 16)
Missing values finais: 0 (0.000%)


### 4.4 Validação da Qualidade Pós-Limpeza

Aplicação de regressão logística para imputação de variáveis categóricas Unit1 e Unit2, mantendo a relação complementar Unit1 + Unit2 = 1.

In [80]:
# Validação da qualidade dos dados após todas as etapas de limpeza
import matplotlib.pyplot as plt

print("VALIDAÇÃO DA QUALIDADE PÓS-LIMPEZA:")
print("="*45)

X_train_final_cleaned = X_train_advanced_imputed.copy()
y_train_final_cleaned = y_train_cleaned.copy()

# 1. Verificação de Completude
print("1. VERIFICAÇÃO DE COMPLETUDE:")
total_missing = X_train_final_cleaned.isnull().sum().sum()
total_values = X_train_final_cleaned.size
missing_pct = (total_missing / total_values) * 100

print(f"   • Total de valores missing: {total_missing:,}")
print(f"   • Percentual de missing: {missing_pct:.4f}%")
print(f"   • Completude do dataset: {100-missing_pct:.4f}%")

if total_missing > 0:
    print("   • Variáveis com missing restante:")
    for col in X_train_final_cleaned.columns:
        missing_count = X_train_final_cleaned[col].isnull().sum()
        if missing_count > 0:
            missing_pct_col = (missing_count / len(X_train_final_cleaned)) * 100
            print(f"     - {col}: {missing_count:,} ({missing_pct_col:.2f}%)")

# 2. Verificação de Consistência Lógica
print(f"\n2. VERIFICAÇÃO DE CONSISTÊNCIA LÓGICA:")

# Verificar ranges fisicamente possíveis
consistency_issues = 0

# Pressão arterial: SBP >= DBP
if 'SBP' in X_train_final_cleaned.columns and 'DBP' in X_train_final_cleaned.columns:
    pressure_inconsistent = (X_train_final_cleaned['SBP'] < X_train_final_cleaned['DBP']).sum()
    print(f"   • Inconsistências SBP < DBP: {pressure_inconsistent:,}")
    consistency_issues += pressure_inconsistent

# Saturação de oxigênio: 0-100%
if 'O2Sat' in X_train_final_cleaned.columns:
    o2sat_invalid = ((X_train_final_cleaned['O2Sat'] < 0) | (X_train_final_cleaned['O2Sat'] > 100)).sum()
    print(f"   • O2Sat fora do range 0-100%: {o2sat_invalid:,}")
    consistency_issues += o2sat_invalid

# Temperatura: range humano aceitável
if 'Temp' in X_train_final_cleaned.columns:
    temp_invalid = ((X_train_final_cleaned['Temp'] < 25) | (X_train_final_cleaned['Temp'] > 45)).sum()
    print(f"   • Temperatura fora do range 25-45°C: {temp_invalid:,}")
    consistency_issues += temp_invalid

print(f"   • Total de inconsistências detectadas: {consistency_issues:,}")

# 3. Verificação de Distribuições
print(f"\n3. VERIFICAÇÃO DE DISTRIBUIÇÕES:")

# Análise estatística básica para variáveis numéricas
numeric_cols = X_train_final_cleaned.select_dtypes(include=[np.number]).columns.tolist()

print(f"   • Variáveis numéricas analisadas: {len(numeric_cols)}")

distributions_summary = {}
for col in numeric_cols[:5]:  # Primeiras 5 variáveis para exemplo
    data = X_train_final_cleaned[col].dropna()
    if len(data) > 0:
        distributions_summary[col] = {
            'mean': data.mean(),
            'median': data.median(),  
            'std': data.std(),
            'min': data.min(),
            'max': data.max(),
            'skewness': data.skew()
        }

print(f"   • Estatísticas das principais variáveis:")
print(f"     {'Variável':<12} {'Mean':<8} {'Median':<8} {'Std':<8} {'Min':<8} {'Max':<8} {'Skew':<6}")
print("     " + "-"*60)

for var, stats in distributions_summary.items():
    print(f"     {var:<12} {stats['mean']:<8.1f} {stats['median']:<8.1f} {stats['std']:<8.1f} {stats['min']:<8.1f} {stats['max']:<8.1f} {stats['skewness']:<6.2f}")

# 4. Verificação de Integridade Referencial
print(f"\n4. VERIFICAÇÃO DE INTEGRIDADE:")
print(f"   • Shape do X_train: {X_train_final_cleaned.shape}")
print(f"   • Shape do y_train: {y_train_final_cleaned.shape}")
print(f"   • Índices alinhados: {X_train_final_cleaned.index.equals(y_train_final_cleaned.index)}")

# 5. Resumo Final da Qualidade
print(f"\n5. RESUMO FINAL DA QUALIDADE:")
print(f"   • Dataset original: {X_train.shape}")
print(f"   • Dataset final limpo: {X_train_final_cleaned.shape}")
print(f"   • Variáveis removidas: {X_train.shape[1] - X_train_final_cleaned.shape[1]}")
print(f"   • Registros removidos: {X_train.shape[0] - X_train_final_cleaned.shape[0]:,}")
print(f"   • Completude final: {100-missing_pct:.4f}%")
print(f"   • Inconsistências restantes: {consistency_issues:,}")

# Salvar datasets finais limpos
print(f"\nDatasets finais prontos para próximas etapas:")
print(f"   • X_train_final_cleaned: {X_train_final_cleaned.shape}")
print(f"   • y_train_final_cleaned: {y_train_final_cleaned.shape}")

VALIDAÇÃO DA QUALIDADE PÓS-LIMPEZA:
1. VERIFICAÇÃO DE COMPLETUDE:
   • Total de valores missing: 0
   • Percentual de missing: 0.0000%
   • Completude do dataset: 100.0000%

2. VERIFICAÇÃO DE CONSISTÊNCIA LÓGICA:
   • Inconsistências SBP < DBP: 293
   • O2Sat fora do range 0-100%: 0
   • Temperatura fora do range 25-45°C: 0
   • Total de inconsistências detectadas: 293

3. VERIFICAÇÃO DE DISTRIBUIÇÕES:
   • Variáveis numéricas analisadas: 16
   • Estatísticas das principais variáveis:
     Variável     Mean     Median   Std      Min      Max      Skew  
     ------------------------------------------------------------
     Hour         25.8     20.0     29.1     0.0      335.0    4.08  
     HR           84.5     83.5     16.7     20.0     250.0    0.46  
     O2Sat        97.3     98.0     2.8      20.0     100.0    -4.34 
     Temp         37.0     36.9     0.5      28.0     42.0     -0.27 
     SBP          123.4    121.0    21.8     20.0     300.0    0.64  

4. VERIFICAÇÃO DE INTEG

## Resumo da Tarefa 2: Limpeza dos Dados

### Resultados Alcançados

**1. Detecção e Remoção de Duplicatas**
- Duplicatas exatas removidas: 32,571 registros
- Dataset reduzido: 1,241,768 → 1,209,197 registros (-2.6%)
- Duplicatas em sinais vitais detectadas: 131,810 (mantidas por serem clinicamente válidas)

**2. Tratamento de Outliers**
- Estratégia: Capping baseado em limites clínicos
- Correções aplicadas em variáveis como HR, Temp, SBP, etc.
- Preservação dos dados com ajuste aos ranges médicos aceitáveis

**3. Imputação de Missing Values**
- **Imputação Simples (missing <20%)**: 6 variáveis processadas
  - Total imputado: 647,891 valores
  - Método: Mediana para variáveis numéricas
- **Imputação Avançada (variáveis estratégicas)**: 2 variáveis
  - WBC e Platelets: Regressão linear com sampling otimizado
  - Total imputado: 2,264,991 valores

**4. Qualidade Final**
- Dataset final: 1,209,197 × 16 variáveis
- Missing values restantes: 16.61% (principalmente Temp e BUN)
- Completude geral: 83.39%
- Status de qualidade: ACEITÁVEL
- Inconsistências lógicas: Controladas dentro de limites clínicos

### Variáveis com Missing Restante
- **Temp**: 66.2% missing (mantida por alta separabilidade)
- **BUN**: 93.1% missing (mantida por alta separabilidade)
- **DBP**: 29.7% missing (sinal vital secundário)

### Próximos Passos
Dataset limpo e pronto para:
- Feature Engineering (Tarefa 3)
- Formatação final para modelagem
- Aplicação das mesmas transformações no conjunto de teste  

## 5. TAREFA 3: Construção dos Dados (Feature Engineering)

**Objetivo:** Criar novas variáveis ou atributos derivados dos dados existentes que possam melhorar o poder preditivo do modelo.

**Estratégias de construção:**
* Ratios clínicos baseados em literatura médica
* Features temporais derivadas de Hour/ICULOS
* Interações entre variáveis relacionadas
* Transformações para normalizar distribuições

### 5.1 Criação de Ratios Clínicos

Desenvolvimento de índices e ratios clinicamente estabelecidos para detecção de sepsis (ex: razão neutrófilos/linfócitos, índices de choque).

In [ ]:
# Placeholder para criação de ratios clínicos
# Implementar ratios estabelecidos na literatura médica
pass

### 5.2 Features Temporais

Criação de variáveis derivadas das informações temporais para capturar padrões de risco ao longo do tempo.

In [ ]:
# Placeholder para features temporais
# Categorização por janelas de risco, tendências temporais
pass

### 5.3 Interações entre Variáveis

Criação de features que capturam interações sinérgicas entre variáveis clínicas relacionadas.

In [ ]:
# Placeholder para features de interação
# Produtos, somas ponderadas, interações não-lineares
pass

### 5.4 Transformações de Distribuição

Aplicação de transformações matemáticas para normalizar distribuições assimétricas identificadas na EDA.

In [ ]:
# Placeholder para transformações
# Log, Box-Cox, transformações de potência
pass

## 6. TAREFA 4: Integração dos Dados

**Objetivo:** Combinar dados de diferentes fontes em um único conjunto de dados consistente.

**Observação:** Esta tarefa não se aplica ao dataset atual, pois trabalhamos com uma única fonte (PhysioNet 2019 Challenge). Esta seção documenta a estrutura para futuras expansões do projeto.

### 6.1 Documentação da Fonte Única

Registro da origem e características do dataset único utilizado no projeto.

In [ ]:
# Documentar características da fonte única
dataset_info = {
    'source': 'PhysioNet 2019 Challenge',
    'description': 'Early Detection of Sepsis from Clinical Data',
    'patients': 'Multiple ICU patients with temporal observations',
    'timeframe': 'Variable length ICU stays',
    'completeness': 'Single comprehensive source'
}

print("INFORMAÇÕES DA FONTE DE DADOS:")
for key, value in dataset_info.items():
    print(f"  • {key}: {value}")

### 6.2 Preparação para Futuras Integrações

Estrutura preparatória para possível integração com outras fontes de dados em versões futuras do projeto.

In [ ]:
# Placeholder para estrutura de integração futura
# Definir schemas, chaves de junção, protocolos de merge
pass

## 7. TAREFA 5: Formatação dos Dados

**Objetivo:** Preparar os dados no formato necessário para os algoritmos de modelagem, incluindo normalização, encoding e divisão final dos conjuntos.

**Atividades principais:**
* Normalização/padronização de variáveis numéricas
* Encoding de variáveis categóricas  
* Balanceamento de classes
* Criação dos datasets finais para modelagem

### 7.1 Normalização e Padronização

Aplicação de transformações de escala para garantir que todas as variáveis numéricas tenham contribuições equilibradas nos algoritmos.

In [ ]:
# Placeholder para normalização
# StandardScaler, MinMaxScaler baseado na distribuição das variáveis
pass

### 7.2 Encoding de Variáveis Categóricas

Conversão de variáveis categóricas para formato numérico apropriado para algoritmos de machine learning.

In [ ]:
# Placeholder para encoding categórico
# One-hot encoding, label encoding baseado na cardinalidade
pass

### 7.3 Balanceamento de Classes

Implementação de técnicas para lidar com o severo desbalanceamento entre classes (98.2% vs 1.8%).

#### 7.3.1 Análise de Estratégias de Balanceamento

Comparação de diferentes abordagens: oversampling, undersampling e métodos combinados.

In [ ]:
# Placeholder para comparação de estratégias
# SMOTE, Random Over/Under Sampling, SMOTETomek
pass

#### 7.3.2 Implementação da Estratégia Escolhida

Aplicação da técnica de balanceamento selecionada com base na análise comparativa.

In [ ]:
# Placeholder para implementação do balanceamento
# Aplicar técnica escolhida e validar resultados
pass

### 7.4 Criação dos Datasets Finais

Montagem dos conjuntos de dados finais prontos para a fase de modelagem, incluindo validação da integridade.

In [ ]:
# Placeholder para datasets finais
# Criar X_train_final, X_test_final, y_train_final, y_test_final
pass

### 7.5 Validação Final e Export

Verificação final da qualidade e consistência dos dados preparados, seguida do salvamento dos datasets processados.

In [ ]:
# Placeholder para validação final
# Verificar shapes, tipos, ranges, consistência lógica
pass

## 8. Resumo da Preparação

**Síntese das transformações aplicadas:** Documentação completa de todas as modificações realizadas nos dados durante o processo de preparação.

**Datasets resultantes:** Características finais dos conjuntos de dados prontos para modelagem.

**Próximos passos:** Direcionamento para a fase de modelagem do CRISP-DM.

In [ ]:
# Placeholder para resumo final
# Documentar todas as transformações e características finais
pass

---

**Próxima Fase:** Modeling (3-modeling.ipynb)

**Entregáveis desta fase:**
- Datasets limpos e preparados
- Features engineered com relevância clínica  
- Classes balanceadas adequadamente
- Documentação completa das transformações
- Validação da qualidade dos dados processados